In [1]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import wordpunct_tokenize
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from scipy import linalg
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from scipy import spatial
import numpy as np
import pandas as pd


stop_words = set(stopwords.words('english'))

In [2]:
def load_from_csv(filename):
    data_dict = {}
    
    df = pd.read_csv(filename)
    print(len(df))
    df = df.dropna()
    df = df.drop_duplicates()
    print(len(df))
    #print(df.head())
    print(len(df["Product_url"].to_list()))
    print(len(set(df["Product_url"].to_list())))
    #pd.Series(df.documents.values,index = df.df.Product_url).to_dict()
    return pd.Series(df.documents.values,index = df.Product_url).to_dict(), df
        

In [3]:
data_dict, modified_df = load_from_csv("product_Urls_res.csv")


500
259
259
259


In [4]:
len(data_dict)

259

In [5]:
def data_processing(data):    #data is a list of lists
    
    res = []
    asciichars = set(string.printable)
    stemmer = PorterStemmer()
    for document in data.values():
        #print(document)
        document = ''.join(ch for ch in document if ch in asciichars)
        document = re.split("[" + string.punctuation + string.whitespace + "]+", document)
        document = [word for word in document if word.isalpha()]
        document = [word for word in document if len(word) > 1]
        document = [word for word in document if not word in stop_words]
        document = [stemmer.stem(word) for word in document]
        document = [word.lower() for word in document] 
        document = ' '.join(document)
        res.append(document)
        
        
    return res

In [6]:
res = data_processing(data_dict)


In [7]:
def generating_tf_idf(data,vocab = None):
    vectorizer = TfidfVectorizer(stop_words=stop_words, max_df = 0.5,
                                 #max_features = 15,
                                    use_idf = True,
                                    ngram_range=(1, 2),vocabulary = vocab)
    vectors = vectorizer.fit_transform(data)
    
    feature_names = vectorizer.get_feature_names()
    dense = vectors.todense().tolist()
    #df = pd.DataFrame(dense, columns=feature_names)

    return dense,feature_names

In [8]:
feature_df,vocab = generating_tf_idf(res)

In [9]:
def Reduce_Dim_SVD(feature_df,num_features = 100):
    svd = TruncatedSVD(num_features)
    lsa = make_pipeline(svd, Normalizer(copy = False))

    feature_data = lsa.fit_transform(feature_df)
    return feature_data, lsa

In [10]:
feature_data, lsa = Reduce_Dim_SVD(feature_df)


In [11]:
query = "textile good quality"

In [12]:
def VSM_Model(search_query,total_data,vocab, lsa):
    processed_query = data_processing({1:search_query})
    # Normalize query vector
#     query_vocab = []
#     for word in tokenized_search_query:
#         if word not in query_vocab:
#             query_vocab.append(word)
#     query_wc = {}
#     for word in query_vocab:
#         query_wc[word] = tokenized_search_query.count(word)
    query_tf_idf,_ = generating_tf_idf(processed_query,vocab)
    query_reduced_dim = lsa.transform(query_tf_idf)
    #Reduce_Dim_SVD(feature_data, query_tf_idf,5)
    
    print(query_reduced_dim)
    scores_dict = []
    for data in total_data:
        #print(type(data))
        scores_dict.append(1 - spatial.distance.cosine(data, query_reduced_dim[0]))
    print(sorted(scores_dict,reverse = True))
    return (np.argsort(scores_dict))[::-1]
    

In [13]:
search_results = VSM_Model(query,feature_data,vocab,lsa)

[[ 0.49462389  0.34681907  0.18265315 -0.20785513  0.01253184 -0.01576052
  -0.03391984 -0.01499433  0.0662977   0.05203228 -0.01882227 -0.00258591
   0.03406382  0.02123056  0.06215913  0.12100089  0.01196297 -0.258058
   0.16417218  0.02045348  0.03964923 -0.03126231  0.0228239   0.00289922
   0.01363327  0.0120079   0.1031863   0.01517302 -0.02840677  0.1158908
  -0.00654986  0.01391275  0.03237429 -0.02252103 -0.04781861  0.11812531
  -0.05921619  0.10573125 -0.00215787 -0.03144965 -0.06624969  0.01018741
   0.01930798 -0.07524537 -0.12313116 -0.1086052  -0.1453194   0.0914038
   0.06434515 -0.05175151  0.07057264  0.18629226  0.1975627   0.04859801
  -0.11309431  0.00096881 -0.06888426  0.02293272 -0.02880668 -0.06512289
  -0.01326831 -0.10722814 -0.0329462  -0.18210577  0.10084493 -0.07330778
   0.06702203  0.03880938 -0.02060579  0.07457424 -0.0391626  -0.01159104
  -0.03673047 -0.08674816  0.12804236 -0.10313738 -0.0041967   0.03761046
   0.08532188  0.04838619 -0.03418857  0.0

In [15]:
modified_df["Features"] = feature_data.tolist()
modified_df.to_csv("Features_res.csv")
print(len(feature_data))

259


In [16]:
modified_df.iloc[search_results, :][:50].to_csv("search_results.csv",header = False,index = False)

In [17]:
feature_data.tolist()


[[0.08703994348777012,
  -0.07850832405833319,
  0.037956393030882,
  -0.06369115232183373,
  0.06031431960279337,
  0.00110563281766528,
  -0.02243844684782209,
  0.002202077792517754,
  -0.00863649871249746,
  -0.009723997881399814,
  0.023182776501900444,
  0.05929062252442735,
  0.010686653607787129,
  0.05776401187668734,
  0.10069460751653218,
  0.031918344145981964,
  0.01114308003546287,
  -0.048984325929715564,
  -0.008464248759781473,
  0.03146358827991325,
  -0.03129228417930414,
  -0.08884541337572388,
  -0.0349368059649192,
  0.051630644340302935,
  0.04113587440378096,
  0.10007700157604871,
  0.0014794284853158954,
  -0.03798042808122067,
  -0.023673640244940893,
  0.03490274570969755,
  -0.027391529539326564,
  0.00281063780727765,
  -0.019456134149388002,
  -0.05290278787984589,
  -0.019402504642765488,
  0.02092164075514236,
  0.017699808678024266,
  0.036852667489420884,
  -0.06078370835472086,
  0.0007120154357314435,
  -0.058058689707471495,
  0.00458850632818491,


In [ ]:
len(modified_df)

In [ ]:
print(len(search_results))